In [1]:
# import libs
import numpy as np
import pandas as pd
import os
import scipy.io as io

In [2]:
# def function

# 获取csv文件的列名
def getcols(filename):
    data = pd.read_csv(filename, nrows = 2)
    #print(data.columns)
    return data.columns

# 获取文件行数
def getrows(filename):
    cols = getcols(filename)
    col = cols[0]
    data = pd.read_csv(filename, usecols=[col])
    return data.shape[0]

# 数据预处理- step1: 删除无法用于计算的列与含空值、负值、inf值的行
def preprocess2017(data):
    labels = data[' Label']
    # 删除非数值型数据，这些数据暂时不用于聚类处理，但不代表这些数据没用
    dropset = ['Flow ID', ' Source IP', ' Source Port',
       ' Destination IP', ' Destination Port', ' Protocol', ' Timestamp',
           ' Fwd Header Length.1', ' Label']
    # 删除含有大量负数的列
    drop_nega = [' Fwd Header Length', 'Init_Win_bytes_forward',
           ' Init_Win_bytes_backward', ' act_data_pkt_fwd',
       ' min_seg_size_forward']
    # 删除数据中可能出现的非数值型数据列
    drop_ifexist = ['External IP','SimillarHTTP', ' Inbound','Unnamed: 0']
    
    # 如果有list中的columns则删除
    data.drop(data.columns[data.columns.isin(dropset+drop_nega+drop_ifexist)], axis = 1, inplace=True)
    
    data.dropna(how='any', inplace=True)
    print(len(data))
    data.reset_index(inplace=True, drop=True)
    # 将所有数据类型转成float型
    data = data.astype('float')
    
    # 删除数据中inf的值 (需要连续执行两次，不知道原因，但连续执行后就可删除数据中的inf)
    idx_tuple = np.where(data.max(axis=1).values == np.inf)
    idx = list(idx_tuple[0])
   # print(idx)
    data.drop(axis=0, index = idx, inplace = True)
    data.reset_index(inplace=True, drop=True)
    labels.drop(axis=0, index = idx, inplace = True)
    labels.reset_index(inplace=True, drop=True)
   # print(len(data))
    idx_tuple = np.where(data.max(axis=1).values == np.inf)
    idx = list(idx_tuple[0])
    data.drop(axis=0, index = idx, inplace = True)
    data.reset_index(inplace=True, drop=True)
    labels.drop(axis=0, index = idx, inplace = True)
    labels.reset_index(inplace=True, drop=True)
   # print(len(data))

    # 删除数据中含负数的数据条数
    idx_tuple = np.where(data.min(axis=1).values < 0)
    idx = list(idx_tuple[0])
    data.drop(axis=0, index = idx, inplace = True)
    data.reset_index(inplace=True, drop=True)
    labels.drop(axis=0, index = idx, inplace = True)
    labels.reset_index(inplace=True, drop=True)
   # print(len(data))
    return data,labels

def preprocess2019(data, cols, labelnum, label):
    data.dropna(how='any', inplace=True)
    
    # print(data.columns)
    data.drop(cols[0:2],axis = 1, inplace = True)
    
    # 删除非数值型数据，这些数据暂时不用于聚类处理，但不代表这些数据没用
    dropset = ['Flow ID', ' Source IP', ' Source Port',
       ' Destination IP', ' Destination Port', ' Protocol', ' Timestamp',
           ' Fwd Header Length.1', 'SimillarHTTP', ' Inbound']
    
    # 删除含有大量负数的列
    drop_nega = [' Fwd Header Length', 'Init_Win_bytes_forward',
           ' Init_Win_bytes_backward', ' act_data_pkt_fwd',
       ' min_seg_size_forward']
    data.drop(dropset, axis=1,inplace=True)
    data.drop(drop_nega, axis=1,inplace=True)
    
    # 将标签替换为数字
    data.replace(to_replace=label, value=labelnum, inplace=True)
    
    # 将所有数据类型转成float型
    data = data.astype('float')
    
    # 删除数据中inf的值
    idx_tuple = np.where(data.max(axis=1).values == np.inf)
    idx = list(idx_tuple[0])
#    print(len(idx))
    data.drop(axis=0, index = idx, inplace = True)
    data.reset_index(inplace=True, drop=True)
    
    idx_tuple = np.where(data.max(axis=1).values == np.inf)
    idx = list(idx_tuple[0])
#    print(len(idx))
    data.drop(axis=0, index = idx, inplace = True)
    data.reset_index(inplace=True, drop=True)
    
    # 删除数据中含负数的数据条数
    idx_tuple = np.where(data.min(axis=1).values < 0)
    idx = list(idx_tuple[0])
    data.drop(axis=0, index = idx, inplace = True)
    data.reset_index(inplace=True, drop=True)
    
    return data

In [3]:
# 读取攻击数据文件
attackfilesdir2019 = '/mnt/hgfs/linuxfile/2019/'
attackfilesdir2017 = '/mnt/hgfs/linuxfile/2017/'
attackfiles2019 = os.listdir(attackfilesdir2019)
attackfiles2017 = os.listdir(attackfilesdir2017)
attackfiles2019, attackfiles2017

(['DrDoS_DNS.csv',
  'DrDoS_LDAP.csv',
  'DrDoS_MSSQL.csv',
  'DrDoS_NetBIOS.csv',
  'DrDoS_NTP.csv',
  'DrDoS_SNMP.csv',
  'DrDoS_SSDP.csv',
  'DrDoS_UDP.csv',
  'Syn.csv',
  'TFTP.csv',
  'UDPLag.csv'],
 ['Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv',
  'Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv',
  'Friday-WorkingHours-Morning.pcap_ISCX.csv',
  'Monday-WorkingHours.pcap_ISCX.csv',
  'Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv',
  'Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv',
  'Tuesday-WorkingHours.pcap_ISCX.csv',
  'Wednesday-workingHours.pcap_ISCX.csv'])

In [18]:
attackfile2019 = attackfilesdir2019+attackfiles2019[4]
attack = pd.read_csv(attackfile2019, usecols=[' Timestamp', ' Label'])

In [23]:
attackfile2019

'/mnt/hgfs/linuxfile/2019/DrDoS_NTP.csv'

In [24]:
attack

,Timestamp,Label
0,2018-12-01 09:17:11.183810,DrDoS_NTP
1,2018-12-01 09:17:11.205636,DrDoS_NTP
2,2018-12-01 09:17:12.634569,BENIGN
3,2018-12-01 09:17:13.458370,BENIGN
4,2018-12-01 09:17:13.470913,BENIGN
...,...,...
1217002,2018-12-01 10:51:39.600556,DrDoS_NTP
1217003,2018-12-01 10:51:39.635018,DrDoS_NTP
1217004,2018-12-01 10:51:39.685553,DrDoS_NTP
1217005,2018-12-01 10:51:39.721301,DrDoS_NTP


In [27]:
attack[attack[' Label'] == 'DrDoS_NTP']

,Timestamp,Label
0,2018-12-01 09:17:11.183810,DrDoS_NTP
1,2018-12-01 09:17:11.205636,DrDoS_NTP
9,2018-12-01 09:17:16.230610,DrDoS_NTP
10,2018-12-01 09:17:16.404689,DrDoS_NTP
11,2018-12-01 09:17:16.404868,DrDoS_NTP
...,...,...
1217002,2018-12-01 10:51:39.600556,DrDoS_NTP
1217003,2018-12-01 10:51:39.635018,DrDoS_NTP
1217004,2018-12-01 10:51:39.685553,DrDoS_NTP
1217005,2018-12-01 10:51:39.721301,DrDoS_NTP
